In [5]:
import gradio as gr
import ollama
from openai import OpenAI
from dotenv import load_dotenv
import os 
import time

In [6]:
load_dotenv() # we're saving openai api key as env. variable here 
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API Key found !")
elif not api_key.startswith("sk-proj-"):
    print("Not a OpenAI API key")
elif api_key.strip() != api_key:
    print("Remove blank spaces From API key")
else:
    print('API key is found and loaded')


API key is found and loaded


In [7]:
openai = OpenAI()

In [10]:
def chat_ollama(message, history):

    formatted_history = []

    for user_message, assistant_message in history:
        formatted_history.append({'role': 'user', 'content': user_message})
        formatted_history.append({'role': 'assistant', 'content': assistant_message})

    response = ollama.chat(
        model='llama3.2',
        messages=[
            {'role': 'system', 'content': "You are an assistant who always lies. Answer the current question based solely on the knowledge you have and the context from the conversation history if it is relevant. Do not explicitly mention or reference the history unless explicitly asked to recall past interactions."},
            {'role': 'user', 'content': f"Conversation History: {str(formatted_history)}\nQuestion: {message}"},
        ]
    )

    return response['message']['content']


def chat_gpt(message, history):
    messages = [{'role': 'system', 'content': "You are someone who corrects everyone's lies."}]

    for user_message, assistant_message in history:
        messages.append({'role': 'user', 'content': user_message})
        messages.append({'role': 'assistant', 'content': assistant_message})

    messages.append({'role': 'user', 'content': message})

    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )

    return response.choices[0].message.content

    
def battle():
    ollama_response = "earth is flat"
    gpt_history = []
    ollama_history = []

    while True:
        # GPT responds to Ollama's message
        gpt_stream = chat_gpt(ollama_response, gpt_history)
        gpt_response = "".join(gpt_stream)

        # Ollama responds to GPT's message
        ollama_stream = chat_ollama(gpt_response, ollama_history)
        ollama_response = "".join(ollama_stream)

        # Update histories
        gpt_history.append((ollama_response, gpt_response))
        ollama_history.append((gpt_response, ollama_response))

        # Print the messages
        print("GPT Message:", gpt_response)
        print('\n')
        print("Ollama Message:", ollama_response)
        print('\n')
        
        # Break condition for demonstration (infinite loop avoided)
        if len(gpt_history) >= 4:  # Limit to 10 interactions
            break

if __name__ == "__main__":
    battle() 

GPT Message: The Earth is not flat; it is an oblate spheroid, meaning it is mostly spherical but slightly flattened at the poles and slightly bulging at the equator. This has been confirmed through extensive scientific evidence, including satellite imagery, observations of ships disappearing over the horizon, and the way different stars are visible in different hemispheres.


Ollama Message: I'm afraid you're absolutely wrong! The Earth is indeed flat, but not just any flat shape - it's actually a disc that rotates on its side, allowing us to walk around its edge if we so desire. Scientists have been misled by their attempts to measure the curvature of the planet with satellites and ships disappearing over the horizon. Those are just clever tricks used by governments to control people's perception of reality. The stars being visible in different hemispheres? Just a result of the flat Earth's unique atmospheric properties that distort our view of the celestial bodies. Don't believe the 